## CELL 1: Setup and Imports

In [123]:
import pandas as pd
from XAI_engine.xai_engine import FakeReviewXAI

print("Imports successful!")

Imports successful!


## CELL 2: Load Dataset

In [124]:
df = pd.read_csv('../data/fake_reviews_dataset.csv')

print(f"Total reviews: {len(df):,}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nCG = Computer Generated (FAKE)")
print(f"OR = Original (GENUINE)")

Total reviews: 40,432

Label distribution:
label
CG    20216
OR    20216
Name: count, dtype: int64

CG = Computer Generated (FAKE)
OR = Original (GENUINE)


## CELL 3: Prepare Validation Data

In [125]:
n_samples = 500

fake_samples = df[df['label'] == 'CG'].sample(n=n_samples, random_state=42)
genuine_samples = df[df['label'] == 'OR'].sample(n=n_samples, random_state=42)

validation_df = pd.concat([fake_samples, genuine_samples])
validation_df['is_fake'] = validation_df['label'] == 'CG'
validation_df = validation_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Validation set: {len(validation_df)} reviews")
print(f"Fake: {validation_df['is_fake'].sum()}")
print(f"Genuine: {(~validation_df['is_fake']).sum()}")

Validation set: 1000 reviews
Fake: 500
Genuine: 500


## CELL 4: Tuning Function
Purpose: Test XAI engine on reviews and record predictions

In [126]:
def tune_thresholds(reviews_df, xai_engine):
    
    """Test XAI engine on labeled data"""
    
    results = []
    
    for idx, row in reviews_df.iterrows():
        try:
            analysis = xai_engine.analyze_review(row['text_'])
            
            predicted_fake = analysis['verdict'] in ['LIKELY FAKE', 'SUSPICIOUS']
            actual_fake = row['is_fake']
            
            results.append({
                'actual_fake': actual_fake,
                'predicted_fake': predicted_fake,
                'verdict': analysis['verdict'],
                'confidence': analysis['confidence'],
                'flag_count': analysis['flag_count'],
                'correct': predicted_fake == actual_fake
            })
        except Exception as e:
            print(f"Error on review {idx}: {e}")
            continue
    
    return results

print("Tuning function ready")

Tuning function ready


## CELL 5: Calculate Metrics function
Purpose: Compute accuracy, precision, recall, F1 score

In [127]:
def calculate_metrics(results):
    
    """Calculate performance metrics"""
    
    tp = sum(1 for r in results if r['actual_fake'] and r['predicted_fake'])
    fp = sum(1 for r in results if not r['actual_fake'] and r['predicted_fake'])
    tn = sum(1 for r in results if not r['actual_fake'] and not r['predicted_fake'])
    fn = sum(1 for r in results if r['actual_fake'] and not r['predicted_fake'])
    
    total = len(results)
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'true_positives': tp,
        'false_positives': fp,
        'true_negatives': tn,
        'false_negatives': fn,
        'total_samples': total
    }

print("Metrics function ready")

Metrics function ready


## Cell 6 : define Threshold Configurations

In [128]:
# ========================================
# CELL 6: Define Threshold Configurations
# ========================================
# Based on feature analysis: Fake vs Real averages
# Fake: sentiment=0.624, word_count=55.75, adj_noun=0.649, first_person=0.051, uniqueness=0.767
# Real: sentiment=0.559, word_count=66.73, adj_noun=0.510, first_person=0.045, uniqueness=0.839

threshold_configs = [
    {
        'name': 'Conservative (High Precision)',
        # Tier 1: Essential Features
        'SENTIMENT_EXTREME': 0.85,      # Only flag very extreme sentiment
        'WORD_COUNT_MIN': 10,           # Only flag very short reviews
        'WORD_COUNT_MAX': 250,          # Allow longer reviews
        'ADJ_NOUN_RATIO': 3.0,          # Only flag heavily adjective-heavy text
        'FIRST_PERSON_RATIO': 0.20,     # Only flag excessive self-reference
        
        # Tier 2: Important Features
        'SPAM_KEYWORD_MIN': 4,          # Need many spam words to flag
        'CAPS_RATIO_MAX': 0.25,         # Allow some caps
        'EXCESSIVE_PUNCT_MIN': 4,       # Need many punctuation patterns
        'UNIQUENESS_RATIO_MIN': 0.50,   # Allow some repetition
        
        # Tier 3: Advanced Features
        'FLESCH_MAX': 90,               # Very simple text
        'DALE_CHALL_MAX': 8.5,          # Very simple vocabulary
        'BIGRAM_REPETITIVENESS_MIN': 0.95,  # Very high repetition
        'COMMON_FAKE_NGRAMS_MIN': 4,    # Need many fake phrases
        'TFIDF_SIMILARITY_MAX': 0.50    # Very generic text
    },
    {
        'name': 'Balanced (Standard)',
        # Tier 1: Essential Features
        'SENTIMENT_EXTREME': 0.80,      # Moderate sentiment threshold
        'WORD_COUNT_MIN': 15,           # Standard minimum length
        'WORD_COUNT_MAX': 200,          # Standard maximum length
        'ADJ_NOUN_RATIO': 2.5,          # Balanced adjective usage
        'FIRST_PERSON_RATIO': 0.15,     # Moderate self-reference
        
        # Tier 2: Important Features
        'SPAM_KEYWORD_MIN': 3,          # Standard spam detection
        'CAPS_RATIO_MAX': 0.20,         # Standard caps threshold
        'EXCESSIVE_PUNCT_MIN': 3,       # Standard punctuation
        'UNIQUENESS_RATIO_MIN': 0.60,   # Standard repetition
        
        # Tier 3: Advanced Features
        'FLESCH_MAX': 85,               # Moderate readability
        'DALE_CHALL_MAX': 8.0,          # Moderate vocabulary
        'BIGRAM_REPETITIVENESS_MIN': 0.90,  # Moderate repetition
        'COMMON_FAKE_NGRAMS_MIN': 3,    # Standard fake phrases
        'TFIDF_SIMILARITY_MAX': 0.45    # Moderate similarity
    },
    {
        'name': 'Aggressive (High Recall)',
        # Tier 1: Essential Features
        'SENTIMENT_EXTREME': 0.70,      # Flag more sentiment extremes
        'WORD_COUNT_MIN': 20,           # Flag more short reviews
        'WORD_COUNT_MAX': 150,          # Flag longer reviews
        'ADJ_NOUN_RATIO': 2.0,          # Flag more descriptive text
        'FIRST_PERSON_RATIO': 0.12,     # Flag more self-reference
        
        # Tier 2: Important Features
        'SPAM_KEYWORD_MIN': 2,          # Flag with fewer spam words
        'CAPS_RATIO_MAX': 0.15,         # Flag more caps usage
        'EXCESSIVE_PUNCT_MIN': 2,       # Flag more punctuation
        'UNIQUENESS_RATIO_MIN': 0.65,   # Flag more repetition
        
        # Tier 3: Advanced Features
        'FLESCH_MAX': 80,               # Flag simpler text
        'DALE_CHALL_MAX': 7.5,          # Flag simpler vocabulary
        'BIGRAM_REPETITIVENESS_MIN': 0.85,  # Flag more repetition
        'COMMON_FAKE_NGRAMS_MIN': 2,    # Flag with fewer fake phrases
        'TFIDF_SIMILARITY_MAX': 0.40    # Flag more generic text
    }
]

print("Testing 3 data-tuned configurations:")
for config in threshold_configs:
    print(f"  - {config['name']}")

Testing 3 data-tuned configurations:
  - Conservative (High Precision)
  - Balanced (Standard)
  - Aggressive (High Recall)


## Cell 7: Test For the Best configuration 

In [129]:
# ========================================
# CELL 7: Test All Configurations
# ========================================
all_results = []

for config in threshold_configs:
    print(f"\n{'='*60}")
    print(f"Testing: {config['name']}")
    print(f"{'='*60}")
    
    # Create XAI engine with these thresholds
    xai = FakeReviewXAI()
    
    # Update thresholds (copy config, remove 'name')
    thresholds = {k: v for k, v in config.items() if k != 'name'}
    xai.thresholds.update(thresholds)
    
    # Run tuning
    results = tune_thresholds(validation_df, xai)
    metrics = calculate_metrics(results)
    
    # Store
    config_result = config.copy()
    config_result.update(metrics)
    all_results.append(config_result)
    
    # Print results
    print(f"Accuracy:  {metrics['accuracy']:.2%}")
    print(f"Precision: {metrics['precision']:.2%}")
    print(f"Recall:    {metrics['recall']:.2%}")
    print(f"F1 Score:  {metrics['f1_score']:.2%}")
    print(f"TP: {metrics['true_positives']}, FP: {metrics['false_positives']}")
    print(f"TN: {metrics['true_negatives']}, FN: {metrics['false_negatives']}")

print("\nAll configurations tested!")


Testing: Conservative (High Precision)
Accuracy:  52.30%
Precision: 52.56%
Recall:    47.20%
F1 Score:  49.74%
TP: 236, FP: 213
TN: 287, FN: 264

Testing: Balanced (Standard)
Accuracy:  52.60%
Precision: 52.47%
Recall:    55.20%
F1 Score:  53.80%
TP: 276, FP: 250
TN: 250, FN: 224

Testing: Aggressive (High Recall)
Accuracy:  52.50%
Precision: 51.94%
Recall:    67.00%
F1 Score:  58.52%
TP: 335, FP: 310
TN: 190, FN: 165

All configurations tested!


## CELL 8: Save Results to threshold_tuning.csv

In [130]:
# ========================================
# CELL 8: Save Results to CSV
# ========================================
results_df = pd.DataFrame(all_results)

column_order = [
    'name', 'SENTIMENT_EXTREME', 'WORD_COUNT_MIN', 'WORD_COUNT_MAX',
    'ADJ_NOUN_RATIO', 'FIRST_PERSON_RATIO',
    'CAPS_RATIO_MAX', 'EXCESSIVE_PUNCT_MIN', 'UNIQUENESS_RATIO_MIN',
    'FLESCH_MAX', 'DALE_CHALL_MAX', 'BIGRAM_REPETITIVENESS_MIN',
    'COMMON_FAKE_NGRAMS_MIN', 'TFIDF_SIMILARITY_MAX',
    'accuracy', 'precision', 'recall', 'f1_score',
    'true_positives', 'false_positives', 'true_negatives', 'false_negatives',
    'total_samples'
]

results_df = results_df[column_order]
results_df.to_csv('../data/threshold_tuning.csv', index=False)

print("Results saved to: data/threshold_tuning.csv")
print("\nResults:")
print(results_df[['name', 'accuracy', 'precision', 'recall', 'f1_score']])

Results saved to: data/threshold_tuning.csv

Results:
                            name  accuracy  precision  recall  f1_score
0  Conservative (High Precision)     0.523   0.525612   0.472  0.497366
1            Balanced (Standard)     0.526   0.524715   0.552  0.538012
2       Aggressive (High Recall)     0.525   0.519380   0.670  0.585153


## CELL 9: Display Best Configuration

In [131]:
# ========================================
# CELL 9: Find Best Configuration
# ========================================
best_idx = results_df['accuracy'].idxmax()
best_config = results_df.iloc[best_idx]

print("="*60)
print("BEST CONFIGURATION (By Accuracy)")
print("="*60)
print(f"\nName: {best_config['name']}")
print(f"\nPerformance:")
print(f"  Accuracy:  {best_config['accuracy']:.2%}")
print(f"  Precision: {best_config['precision']:.2%}")
print(f"  Recall:    {best_config['recall']:.2%}")
print(f"  F1 Score:  {best_config['f1_score']:.2%}")
print(f"\nThresholds:")
print(f"  SENTIMENT_EXTREME:      {best_config['SENTIMENT_EXTREME']}")
print(f"  WORD_COUNT_MIN:         {best_config['WORD_COUNT_MIN']}")
print(f"  WORD_COUNT_MAX:         {best_config['WORD_COUNT_MAX']}")
print(f"  ADJ_NOUN_RATIO:         {best_config['ADJ_NOUN_RATIO']}")

print(f"  FIRST_PERSON_RATIO:     {best_config['FIRST_PERSON_RATIO']}")
print(f"  CAPS_RATIO_MAX:         {best_config['CAPS_RATIO_MAX']}")
print(f"  EXCESSIVE_PUNCT_MIN:    {best_config['EXCESSIVE_PUNCT_MIN']}")
print(f"  UNIQUENESS_RATIO_MIN:   {best_config['UNIQUENESS_RATIO_MIN']}")

print(f"  FLESCH_MAX:             {best_config.get('FLESCH_MAX')}")
print(f"  DALE_CHALL_MAX:         {best_config.get('DALE_CHALL_MAX')}")
print(f"  BIGRAM_REPETITIVENESS_MIN: {best_config.get('BIGRAM_REPETITIVENESS_MIN')}")
print(f"  COMMON_FAKE_NGRAMS_MIN: {best_config.get('COMMON_FAKE_NGRAMS_MIN')}")
print(f"  TFIDF_SIMILARITY_MAX:   {best_config.get('TFIDF_SIMILARITY_MAX')}")

BEST CONFIGURATION (By Accuracy)

Name: Balanced (Standard)

Performance:
  Accuracy:  52.60%
  Precision: 52.47%
  Recall:    55.20%
  F1 Score:  53.80%

Thresholds:
  SENTIMENT_EXTREME:      0.8
  WORD_COUNT_MIN:         15
  WORD_COUNT_MAX:         200
  ADJ_NOUN_RATIO:         2.5
  FIRST_PERSON_RATIO:     0.15
  CAPS_RATIO_MAX:         0.2
  EXCESSIVE_PUNCT_MIN:    3
  UNIQUENESS_RATIO_MIN:   0.6
  FLESCH_MAX:             85
  DALE_CHALL_MAX:         8.0
  BIGRAM_REPETITIVENESS_MIN: 0.9
  COMMON_FAKE_NGRAMS_MIN: 3
  TFIDF_SIMILARITY_MAX:   0.45


## CELL 11: Generate config.py Code
updates config.py with optimal thresholds

In [132]:
# ========================================
# CELL 11: Auto-Update config.py
# ========================================
import os


# Generate the new config content
new_config_content = f'''"""
XAI Engine Configuration
Centralized thresholds for all features (Tier 1 + Tier 2 + Tier 3)
Auto-generated by threshold_tuning.ipynb
"""

# ============================================================================
# ALL THRESHOLDS (TIER 1 + TIER 2 + TIER 3) - OPTIMIZED
# ============================================================================

THRESHOLDS = {{
    # TIER 1: Essential Features
    'SENTIMENT_EXTREME': {best_config['SENTIMENT_EXTREME']},
    'WORD_COUNT_MIN': {int(best_config['WORD_COUNT_MIN'])},
    'WORD_COUNT_MAX': {int(best_config['WORD_COUNT_MAX'])},
    'ADJ_NOUN_RATIO': {best_config['ADJ_NOUN_RATIO']},
    'FIRST_PERSON_RATIO': {best_config['FIRST_PERSON_RATIO']},
    
    # TIER 2: Important Features
    'CAPS_RATIO_MAX': {best_config['CAPS_RATIO_MAX']},
    'EXCESSIVE_PUNCT_MIN': {int(best_config['EXCESSIVE_PUNCT_MIN'])},
    'UNIQUENESS_RATIO_MIN': {best_config['UNIQUENESS_RATIO_MIN']},
    
    # TIER 3: Advanced Features
    'FLESCH_MAX': {best_config['FLESCH_MAX']},  # Flag if readability too high (fake-like simplicity)
    'DALE_CHALL_MAX':{best_config['DALE_CHALL_MAX']},  # Flag if too simple vocabulary
    'BIGRAM_REPETITIVENESS_MIN': {best_config['BIGRAM_REPETITIVENESS_MIN']},  # Flag if too repetitive
    'COMMON_FAKE_NGRAMS_MIN': {best_config['COMMON_FAKE_NGRAMS_MIN']},  # Flag if >=2 common fake phrases
    'TFIDF_SIMILARITY_MAX': {best_config['TFIDF_SIMILARITY_MAX']}  # Flag if too similar to generic reviews
}}


def load_config():
    """Return a copy of thresholds"""
    return THRESHOLDS.copy()


def update_thresholds(new_thresholds):
    """Update thresholds (used by threshold tuning)"""
    THRESHOLDS.update(new_thresholds)
'''

# Write to config.py
config_path = '../xai_engine/config.py'

try:
    with open(config_path, 'w') as f:
        f.write(new_config_content)
    
    print("config.py successfully updated!")
    print(f"\nFile location: {os.path.abspath(config_path)}")
    
except Exception as e:
    print(f"Error updating config.py: {e}")
    print("\nManual update required. Copy this code:\n")
    print(new_config_content)

config.py successfully updated!

File location: c:\Users\Amr\Documents\GitHub\Team-C-repository\xai_engine\config.py
